In [2]:
import glob
import amrlib
import pandas as pd
import penman
from penman import constant
from amrlib.graph_processing.annotator import add_lemmas
from amrlib.alignments.rbw_aligner import RBWAligner
from penman.models.noop import NoOpModel
import ast
import pickle
import os

In [25]:
DATASET = "figlang_reddit"#defining the dataset name throughout
#fake-emo, jing_gab, jing_reddit have errors

In [26]:
# df = pd.read_csv(f'jing_reddit_amr/jing_reddit.csv')

In [15]:
# import preprocessor as pro
# for i, j in df.iterrows():
#     df.at[i, 'text'] = pro.clean(j['text']).strip()
#     comments = j['comments'].split("::")
#     process_comments = []
#     for comm in comments:
#         process_comments.append(pro.clean(comm).strip())
#     df.at[i, 'comments'] = "::".join(process_comments)å

In [18]:
df.to_csv("fake-emo_amr/fake-emo.csv", index = False)

In [4]:
amr_list = glob.glob(f"{DATASET}_amr/{DATASET}_amr_csv/{DATASET}*.csv")

In [5]:
len(amr_list)

7012

### Adding dummy source node and COMMENT edges to multiple AMRs and merge into one.

In [8]:
def modify_variables(amr, i):
    """This function takes an AMR as input and modifies the variables of the AMR depending on the serial number
    of the AMR. Here, (i) refers to the (i)th comment on a particular news piece.
    
    Returns: The modifed penman Graph of the AMR string.
    
    Note: This function does not modify the edpidata or metadata of the input AMR. We just modify the variable names 
    in this function. Since our ultimate goal is to merge several AMR graphs, it is highly likely that different
    amrs have the same variable names. Thus to distinguish between variables of different amrs we assign unique names
    to different variables."""

    g = penman.decode(amr)

    g_meta = add_lemmas(amr, snt_key='snt')#adding lemmas , tokens to the AMR string
    
    #create a dictionary for mapping old variables to new variable names

    var, d = list(g.variables()), {}

    for j in range(len(var)):
        d[var[j]] = "c{}-{}".format(i, j)
        
    #modify the variable names of instances, edges, attributes of the original amr graph 
    instances, edges, attributes, epidata = [], [], [], {}
    for source, role, target in g.instances():#modify the instances
        instances.append((d[source], role, target))
    for source, role, target in g.edges():#modify the edges
        edges.append((d[source], role, d[target]))

    for source, role, target in g.attributes():#modify the attributes
        attributes.append((d[source], role, target))

    for (source, role, target) in g.epidata.keys():#modify the attributes
        
        push_pop = g.epidata[(source, role, target)]
        
        modified_epi = []
        for p in push_pop:
            if isinstance(p, penman.layout.Push):  modified_epi.append(penman.layout.Push(d[p.variable]))
            elif isinstance(p, penman.layout.Pop):  modified_epi.append(p)
            else: print(p)
    
        #if the epidata key is either an instance or attribute triple
        if (source, role, target) in g.instances() or (source, role, target) in g.attributes(): 
            epidata[(d[source], role, target)] = modified_epi
        
        elif (source, role, target) in g.edges(): 
            epidata[(d[source], role, d[target])] = modified_epi
        else:
            print((source, role, target))
        
    modified  = penman.Graph(instances + edges + attributes)#return the modifies graph 
    
    modified.metadata = g_meta.metadata #using the metadata from the original graph
    
    modified.epidata = epidata #using the epidata from the original graph -- name changed

    assert len(eval(modified.metadata['lemmas']))==len(eval(modified.metadata['tokens'])), "Length of tokens must be equal to lemmas"

    return modified

### Change variables in all AMRs for coreference resolution.

In [14]:
lv, ignored = 1, []
for q in amr_list:
    df = pd.read_csv(q)
    #removing any comment with null amr - possible case when language is not english
    df = df.dropna()
    print("Processing", q, end = " ")
    # modify all Comment AMRs belonging to one news article
    modified_amr_list = []
    try:
        for i, j in df.iterrows():
            var_mod = modify_variables(j['amr'], i+1)
            name = q[q.rfind('/')+1:q.rfind('.')]
            modified_amr_list.append(var_mod)
        
        dst_dir = f'{DATASET}_amr/{DATASET}_amr_coref/{name}.amr.penman'
        os.makedirs(os.path.dirname(dst_dir), exist_ok=True)
        penman.dump(modified_amr_list, dst_dir, model = NoOpModel())
    
    except AssertionError:
        print("**************Ignoring the file {}******************".format(q))
        ignored.append(q)

    except:
        print("**************Ignoring the file decode error{}******************".format(q))
        ignored.append(q)

    lv+=1 
    print("Done", lv, len(modified_amr_list))
print("Files ignored:\n{}".format(ignored))
print("{} filed ignored".format(len(ignored)))

Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_3860.csv Done 2 9
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_5013.csv Done 3 1
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_4260.csv **************Ignoring the file decode errorfake-emo_amr/fake-emo_amr_csv/fake-emo_4260.csv******************
Done 4 0
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_2076.csv Done 5 1
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_2328.csv Done 6 1
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_4284.csv Done 7 14
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_2996.csv **************Ignoring the file decode errorfake-emo_amr/fake-emo_amr_csv/fake-emo_2996.csv******************
Done 8 10
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_4476.csv **************Ignoring the file decode errorfake-emo_amr/fake-emo_amr_csv/fake-emo_4476.csv******************
Done 9 5
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_5410.csv Done 10 1
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_3215.

ignoring epigraph data for duplicate triple: ('s', ':ARG1', 'a')
ignoring epigraph data for duplicate triple: ('s', ':ARG1', 'a')


**************Ignoring the file decode errorfake-emo_amr/fake-emo_amr_csv/fake-emo_4895.csv******************
Done 30 6
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_3481.csv Done 31 1
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_6005.csv **************Ignoring the file decode errorfake-emo_amr/fake-emo_amr_csv/fake-emo_6005.csv******************
Done 32 2
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_1108.csv **************Ignoring the file decode errorfake-emo_amr/fake-emo_amr_csv/fake-emo_1108.csv******************
Done 33 2
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_207.csv Done 34 6
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_5909.csv Done 35 1
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_2390.csv Done 36 7
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_872.csv **************Ignoring the file decode errorfake-emo_amr/fake-emo_amr_csv/fake-emo_872.csv******************
Done 37 12
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_2073.csv Done 38 1


ignoring epigraph data for duplicate triple: ('p3', ':ARG0', 'p4')
ignoring epigraph data for duplicate triple: ('p3', ':ARG0', 'p4')


**************Ignoring the file decode errorfake-emo_amr/fake-emo_amr_csv/fake-emo_4086.csv******************
Done 39 13
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_275.csv Done 40 1
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_5185.csv Done 41 1
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_1371.csv **************Ignoring the file decode errorfake-emo_amr/fake-emo_amr_csv/fake-emo_1371.csv******************
Done 42 5
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_1163.csv Done 43 1
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_3626.csv Done 44 1
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_5831.csv **************Ignoring the file decode errorfake-emo_amr/fake-emo_amr_csv/fake-emo_5831.csv******************
Done 45 4
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_5743.csv **************Ignoring the file decode errorfake-emo_amr/fake-emo_amr_csv/fake-emo_5743.csv******************
Done 46 8
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_2050.csv 

...

Done 6118 12
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_2728.csv Done 6119 1
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_2540.csv **************Ignoring the file decode errorfake-emo_amr/fake-emo_amr_csv/fake-emo_2540.csv******************
Done 6120 8
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_682.csv Done 6121 11
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_3698.csv Done 6122 1
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_2879.csv **************Ignoring the file decode errorfake-emo_amr/fake-emo_amr_csv/fake-emo_2879.csv******************
Done 6123 15
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_2342.csv **************Ignoring the file decode errorfake-emo_amr/fake-emo_amr_csv/fake-emo_2342.csv******************
Done 6124 0
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_6520.csv Done 6125 1
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_5030.csv **************Ignoring the file decode errorfake-emo_amr/fake-emo_amr_csv/fake-emo_5030.csv*********

ignoring epigraph data for duplicate triple: ('r', ':ARG1', 'c')
ignoring secondary node contexts for 'c3-8'


Done 6131 15
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_5543.csv **************Ignoring the file decode errorfake-emo_amr/fake-emo_amr_csv/fake-emo_5543.csv******************
Done 6132 0
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_5742.csv Done 6133 11
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_2153.csv Done 6134 1
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_3400.csv Done 6135 1
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_2813.csv **************Ignoring the file decode errorfake-emo_amr/fake-emo_amr_csv/fake-emo_2813.csv******************
Done 6136 6
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_1804.csv **************Ignoring the file decode errorfake-emo_amr/fake-emo_amr_csv/fake-emo_1804.csv******************
Done 6137 3
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_84.csv Done 6138 3
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_1563.csv Done 6139 1
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_3023.csv Done 6140 1
Processing fake-emo

ignoring epigraph data for duplicate triple: ('p6', ':ARG0', 't9')
ignoring epigraph data for duplicate triple: ('p6', ':ARG0', 't9')


**************Ignoring the file decode errorfake-emo_amr/fake-emo_amr_csv/fake-emo_6580.csv******************
Done 6145 6
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_6328.csv Done 6146 15
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_4791.csv Done 6147 1
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_1643.csv **************Ignoring the file decode errorfake-emo_amr/fake-emo_amr_csv/fake-emo_1643.csv******************
Done 6148 0
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_3315.csv Done 6149 1
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_5090.csv Done 6150 8
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_3986.csv Done 6151 1
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_5269.csv **************Ignoring the file decode errorfake-emo_amr/fake-emo_amr_csv/fake-emo_5269.csv******************
Done 6152 1
Processing fake-emo_amr/fake-emo_amr_csv/fake-emo_2066.csv **************Ignoring the file decode errorfake-emo_amr/fake-emo_amr_csv/fake-emo_2066.csv**********

### Coreference Resolution

In [27]:
# %run amr_coref/amr_coref.py --dataset twitter15

### Processing the corefernce resolution clusters and adding the dummy node (+ :COMMENT edge)

In [28]:
import json

In [29]:
def add_edge(graph, source, role, target):

    """Function to add an edge between two previously existing nodes in the graph.
    
    Here, source and target node instances already exist in "graph" and we simply add an edge with relation "rel"
    between the two. The purpose of this is to add :COREF and :SAME edges
    
    TODO: Modify the epidata while adding a new edge"""


    edges= [(source, role, target)]#adding the new edge
    edges.extend(graph.edges())

    #modified amr after adding the required edge
    modified = penman.Graph(graph.instances() + edges + graph.attributes())
    return modified

In [30]:
def coreference_edges(merged_amr, name, amr_coref):

    """ Function to add coreference edges to the merged graph. The input is the combined AMR graph of all smaller
    comment AMR graphs."""
    d = amr_coref[name]
    for relation, cluster in d.items():
        var = [i[1] for i in cluster]
        #amr_coref is sorted according to time (i.e. comments appearing first temporally appear before) by default
        source = var[0] #the directed edge will start from the comment appearing first: following temporal fashion
        for target in var[1:]:#add :COREF edges from the source word to all words in the cluster
            added = add_edge(merged_amr, source, ":COREF", target)
            merged_amr = added
    return merged_amr

In [31]:
def concept_merge(modified):
    normalised_graph = normalise_graph(modified)
    word2var = generate_word2var(normalised_graph)
    for word, var in word2var.items():
        head_node = var[0]
        if len(var)>1:
            for j in var[1:]:
                added = add_edge(normalised_graph, head_node, ":SAME", j)
                normalised_graph = added
    return normalised_graph

In [32]:
def generate_word2var(normalised_graph):
    """This function returns a word-to-variableNames dictionary.
    
    A word might be present on multiple nodes. This returns the dictionary storing the nodes for every word."""
    word2var = {}# a dictionary mapping the words to nodes; example: the word 'name' might belong to 2 nodes

    for (source, role, target) in normalised_graph.instances():
        if target in word2var:
            word2var[target].append(source)
        else:
            word2var[target] = [source]
    return word2var

In [33]:
def normalise_graph(modified):

    """A function to convert the concepts in the amr to meaningful form so that we can apply glove embedding later on
    to find their node representations. 
    
    Note: Removing the part after the hyphen (-) in many of the concept names."""
    normalised_instances = []
    for (source, role, target) in modified.instances():
        if "-" in target:#for example: "save-01" concept is converted to "save". 
            normalised_instances.append((source, role, target[:target.rfind("-")]))
        else:
            normalised_instances.append((source, role, target))
    normalised_graph = penman.Graph(normalised_instances + modified.edges() + modified.attributes())
    return normalised_graph

In [34]:
amr_list = glob.glob(f"{DATASET}_amr/{DATASET}_amr_csv/{DATASET}*.csv")

In [35]:
amr_coref = json.load(open(f"{DATASET}_amr/{DATASET}_amr_coref.json", "r"))#load the coref json with cluster information
amr_coref_names = [i[:i.find(".")] for i in list(amr_coref.keys())]
lv=0
for q in amr_list:
    df = pd.read_csv(q)
    #modify all Comment AMRs belonging to one news article
    name = q[q.rfind('/')+1:q.rfind('.')]#name of current file containing the amrs

    if name not in amr_coref_names:
        print("Skipping")
        continue

    #reading the modified amrs passed as input for coreference resolution
    modified_amr_list = penman.load(f"{DATASET}_amr/{DATASET}_amr_coref/{name}.amr.penman", model = NoOpModel())

    #adding dummy node and :COMMENT edges
    instances, edges, attributes = [('d', ':instance', 'dummy')], [], []
    metadata, epidata= {'snt': '', 'lemmas' : [], 'tokens' : []}, {('d', ':instance', 'dummy') : []}
    
    subgraph_list = []

    for graph in modified_amr_list:

        #for now we are ignoring the :coref and :same-as nodes to be a part of any subgraph - later on they can be added
        node_list = [source for source, _, _ in graph.instances()]#maintained for creating subgraphs later on
        subgraph_list.append(node_list)

        edges.append(('d', ':COMMENT', graph.top))
        instances.extend(graph.instances())
        edges.extend(graph.edges())
        attributes.extend(graph.attributes())
        metadata['snt']+= "{} ".format(graph.metadata['snt'])
        metadata['lemmas'].extend(ast.literal_eval(graph.metadata['lemmas']))
        metadata['tokens'].extend(ast.literal_eval(graph.metadata['tokens']))
        #adding epidata for the added edge
        epidata[('d', ':COMMENT', graph.top)] = [penman.layout.Push(graph.top)]
        #Adding the epidata from all amrs
        epidata.update(graph.epidata)

    metadata['tokens'] = json.dumps(metadata['tokens']) #convert metadata tokens to string format
    metadata['lemmas'] = json.dumps(metadata['lemmas'])
    
    #final modified graph consisting of all comment AMRs corresponding to one news piece
    modified = penman.Graph(instances + edges + attributes)
    modified.metadata = metadata
    modified.epidata = epidata

    #adding the coreference edges to the merged amr graph
    modified = coreference_edges(modified, "{}.amr.penman".format(name), amr_coref)
    
    #adding the concept merging edges to the merged amr graph
    try:
        modified = concept_merge(modified)
    except:
        continue

    modified.metadata['subgraphs'] = json.dumps(subgraph_list)#adding the subgraph information
    #here every subgraph corresponds to a particular comment on the news article and the merged amr is the overall combination of all such subgraphs

    # print(penman.encode(modified))
    #storing the final AMR graph (merged + coreference + concept merging)
    dst_dir = f"{DATASET}_amr/{DATASET}_amr_merge/{name}.amr.penman"
    os.makedirs(os.path.dirname(dst_dir), exist_ok=True)
    penman.dump([modified], dst_dir, model = NoOpModel())

    print("Done", lv, q)
    lv+=1

    # break

ignoring epigraph data for duplicate triple: ('c3-1', ':mod', 'c3-0')


Done 0 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_4238.csv
Done 1 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_991.csv
Done 2 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_530.csv
Done 3 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_392.csv
Done 4 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_4041.csv
Done 5 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_1845.csv
Done 6 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_4231.csv
Done 7 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_88.csv
Done 8 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_1389.csv
Done 9 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_1965.csv
Done 10 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_1979.csv
Done 11 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_1853.csv
Done 12 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_3884.csv
Done 13 figlang_reddit_amr/figlang_reddit_amr_csv/fig

ignoring epigraph data for duplicate triple: ('c1-7', ':ARG0', 'c1-9')


93 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_1126.csv
Done 94 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_1986.csv
Done 95 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_2050.csv
Done 96 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_3284.csv
Done 97 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_1327.csv
Done 98 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_479.csv
Done 99 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_3162.csv
Done 100 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_926.csv
Done 101 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_3440.csv
Done 102 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_3450.csv
Done 103 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_2971.csv
Done 104 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_993.csv
Done 105 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_2125.csv
Done 106 figlang_reddit_amr/figlang_redd

ignoring epigraph data for duplicate triple: ('c2-0', ':mod', 'c2-9')
ignoring epigraph data for duplicate triple: ('c2-0', ':ARG1', 'c2-2')
ignoring epigraph data for duplicate triple: ('c1-0', ':mod', 'c1-5')
ignoring epigraph data for duplicate triple: ('c2-0', ':ARG0', 'c2-6')


...

Done 4059 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_1931.csv
Done 4060 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_4099.csv
Done 4061 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_897.csv
Done 4062 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_1643.csv
Done 4063 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_869.csv
Done 4064 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_3489.csv
Done 4065 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_534.csv
Done 4066 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_2328.csv
Done 4067 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_1374.csv
Done 4068 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_840.csv
Done 4069 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_3181.csv
Done 4070 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_1800.csv
Done 4071 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_1076.csv
Done 4072 figlan

ignoring epigraph data for duplicate triple: ('c2-14', ':ARG1', 'c2-10')
ignoring epigraph data for duplicate triple: ('c2-10', ':ARG0', 'c2-8')
ignoring epigraph data for duplicate triple: ('c2-10', ':ARG1', 'c2-6')
ignoring epigraph data for duplicate triple: ('c3-3', ':name', 'c3-4')


Done 4156 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_3227.csv
Done 4157 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_1246.csv
Done 4158 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_4340.csv
Done 4159 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_3127.csv
Done 4160 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_3057.csv
Done 4161 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_968.csv
Done 4162 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_2686.csv
Done 4163 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_8.csv
Done 4164 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_1448.csv
Done 4165 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_1838.csv
Done 4166 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_1426.csv
Done 4167 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_3452.csv
Done 4168 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_3607.csv
Done 4169 figlan

ignoring epigraph data for duplicate triple: ('c3-8', ':mod', 'c3-20')
ignoring epigraph data for duplicate triple: ('c1-3', ':mod', 'c1-6')


Done 4250 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_3572.csv
Done 4251 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_3496.csv
Done 4252 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_3577.csv
Done 4253 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_2047.csv
Done 4254 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_647.csv
Done 4255 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_4189.csv
Done 4256 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_4274.csv
Done 4257 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_1282.csv
Done 4258 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_551.csv
Done 4259 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_3254.csv
Done 4260 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_3113.csv
Done 4261 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_4006.csv
Done 4262 figlang_reddit_amr/figlang_reddit_amr_csv/figlang_reddit_3369.csv
Done 4263 figl